# KostraRegnskap og regionsaggregering

## Leser inn nødvendige pakker
- Vi trenger følgende pakker i dette arket:
    - Kostra
    - arrow (for å lese inn parquet filer)
    - Tidyverse (brukes strengt tatt ikke, men fin å ha med om man vil gjøre noe annet med datane)

In [ ]:
library(Kostra)
library("arrow")
library(tidyverse)

In [ ]:
source('../KostraPakken_redigert/RegionsAggregering.R',echo=TRUE)

In [ ]:
aar <- readline(prompt="Angi årgang:")

## Leser inn nødvendige filer
- **Vi trenger følgende filer for å kjøre KostraRegnskap**
    - Kasseregnskapet 
        - *Gjør om filen til dataframe* 
        - *Fjerner ekstra kolonne som kommer med i regnskapsfilen. Dette bør ikke komme med. Må finne ut hvorfor å lese inn filen uten denne.*
        - *Endrer kontoklasse fra tall til bokstaver*
        - *Skriver ut de første linjene for å konstrollere at filen blir korrekt. Filen skal inneholde kolonner: periode, region, kontoklasse, art, belop, regnskapsomfang.*

In [ ]:
regnskap <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/kostra0a_0c_basis_',aar,'.parquet', sep = ""))
regnskap <- as.data.frame(regnskap)
regnskap <- select(regnskap, -last_col())
regnskap$kontoklasse <- CharacterReCode(NaToZero(regnskap$kontoklasse), c("0", "1"), c("I", "D"))
head(regnskap,2)

- Særbedriftsregnskapet
    - *Gjør om filen til dataframe* 
    - *Endrer kontoklasse fra tall til bokstaver*

In [ ]:
saerbedrift <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/kostra0i_0k_fordelt_',aar,'.parquet', sep = ""))
saerbedrift <- as.data.frame(saerbedrift)
saerbedrift$kontoklasse <- CharacterReCode(NaToZero(saerbedrift$kontoklasse), c("3", "4", "0"), c("D", "I", "I"))
head(saerbedrift,2)

- Konsolidert
    - *Gjør om filen til dataframe* 
    - *Endrer kontoklasse fra tall til bokstaver*

In [ ]:
konsolidert <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/kostra0m_0p_basis_',aar,'.parquet', sep = ""))
konsolidert <- as.data.frame(konsolidert)
konsolidert$kontoklasse <- CharacterReCode(NaToZero(konsolidert$kontoklasse), c("3", "4", "0"), c("D", "I", "I"))
konsolidert$regnskapsomfang <- gsub('C','B',konsolidert$regnskapsomfang)
konsolidert <- select(konsolidert, -last_col())
head(konsolidert,2)

- Artshierarki
    - *Gjør om filen til dataframe* 

In [ ]:
artshierarki <- read_parquet(paste('../../../Testdata/arbeidsdata/KostraRegnskap/artshierarki_',aar,'.parquet', sep = ""))
artshierarki <- as.data.frame(artshierarki)

- Funksjonshierarki
    - *Gjør om filen til dataframe* 

In [ ]:
funksjonshierarki <- read_parquet(paste('../../../Testdata/arbeidsdata/KostraRegnskap/funksjonshierarki_',aar,'.parquet', sep = ""))
funksjonshierarki <- as.data.frame(funksjonshierarki)

- Formler
    - *Gjør om filen til dataframe* 

In [ ]:
formler <- read_parquet(paste('../../../Testdata/arbeidsdata/KostraRegnskap/formler_',aar,'.parquet', sep = ""))
formler <- as.data.frame(formler)

- Stjernetabell
    - *Gjør om filen til dataframe* 
    - *Endrer manglende verdier fra NA til blank*

In [ ]:
stjerne <- read_parquet(paste('../../../Testdata/arbeidsdata/KostraRegnskap/stjernetabell_',aar,'.parquet', sep = ""))
stjerne <- as.data.frame(stjerne)
stjerne[is.na(stjerne)] <- ""

- Nettingshierarki kasse
    - *Gjør om filen til dataframe* 

In [ ]:
nettinger_kasse <- read_parquet(paste('../../../Testdata/arbeidsdata/KostraRegnskap/nettinger_kasse_',aar,'.parquet', sep = ""))
nettinger_kasse <- as.data.frame(nettinger_kasse)

- Nettingshierarki særbedrift
    - *Gjør om filen til dataframe* 

In [ ]:
nettinger_saer <- read_parquet(paste('../../../Testdata/arbeidsdata/KostraRegnskap/nettinger_saer_',aar,'.parquet', sep = ""))
nettinger_saer <- as.data.frame(nettinger_saer)

- **Vi trenger følgende filer for å kjøre aggregering**
    - Regionshierarki
- Det kommer med en ekstra kolonne når tabellen lastes inn her. Det bør fikses. Fjerner siste kolonne foreløpig.
- Skriver ut de to første radene for å kontrollere hierarkiet. Det skal inneholde følgende kolonner:
    - from
    - periode
    - to
    - sign

In [ ]:
kommunehierarki <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/',aar,'/kommunehierarki.parquet', sep = ""))
kommunehierarki <- as.data.frame(kommunehierarki)
kommunehierarki <- select(kommunehierarki, -last_col())

In [ ]:
fylkeskommunehierarki <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/',aar,'/fylkeskommunehierarki.parquet', sep = ""))
fylkeskommunehierarki <- as.data.frame(fylkeskommunehierarki)
fylkeskommunehierarki <- select(fylkeskommunehierarki, -last_col())

In [ ]:
bydelshierarki <- read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/',aar,'/bydelshierarki.parquet', sep = ""))
bydelshierarki <- as.data.frame(bydelshierarki)
bydelshierarki <- select(bydelshierarki, -last_col())

- **Trenger følgende for å fjerne estimerte kommuner**
    - Kommuner som mangler for 0A og 0C

In [ ]:
manglende <-  read_parquet(paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/kostra0a_0c_manglende_',aar,'.parquet', sep = ""))
manglende <- as.data.frame(manglende)

### Info om KostraRegnskaps-funksjonen

In [ ]:
#?KostraRegnskap

## Kostra regnskapsfunksjonen
- Kjører funksjonen KostraRegnskap fra Kostra biblioteket. Dette er den samme funksjonen som er lagt inn i Kompis.
- Funksjonen må skrives med inputdata i følgende rekkefølge: 
    1. Kasseregnskapet
    2. Funksjonshierarki
    3. Artshierarki
    4. Særbedriftsregnskapet
    5. Nettingshierarki særbedrifter
    6. Nettingshierarki kasse
    7. Stjernetabell
    8. Formler

In [ ]:
resultat <- KostraRegnskap(regnskap, funksjonshierarki, artshierarki, saerbedrift, nettinger_saer, nettinger_kasse, stjerne, formler)

**Filter**  
Legger på et filter for å kunne kontrollere output.  
Kontrollerer output ved å sammenligne med verdiene i Kompis.

In [ ]:
#filter(resultat, region == "0301", art == "AGD23", funksjon == "FG2", kontoklasse == 'D')

## Konsern
### Aggregerer regnskapet

- Kjører funksjon HierarchyCompute fra kostra biblioteket. 
- Funksjonen aggregerer regnskapet til følgende (så sant regionsfierarkiet er korrekt):
    - EAK (Landet - kommuner)
    - EAKUO (Landet uten Oslo - kommuner)
    - EKAXX (Fylker)
    - EKG (kostragrupper)
    - EAFK (Landet - fylkeskommuner)
    - EAFKUO (Landet uten Oslo - fylkeskommuner)
    - EAFKXX (Landsdeler)
    - EAB (Bydeler)  
<br>
<br>
- Vi lager kun aggregerte størrelser på konserntall. Filtrerer derfor bort kasse før aggregeringen. 

In [ ]:
regionshierarki <- rbind(kommunehierarki, fylkeskommunehierarki, bydelshierarki)

In [ ]:
konsern <- filter(resultat, regnskapsomfang == "A")

In [ ]:
aggregert <- KostraRegnskapRegionAggregering(data=konsern, hierarki=regionshierarki)

In [ ]:
aggregert$periode <- '2021'

In [ ]:
konsern_aggregert <- rbind(konsern, aggregert)

### Fjerner estimerte kommuner
- Før filen kan lagres til mappe må estimerte kommuner fjernes. Vi publiserer ikke tall for estimerte kommuner.

In [ ]:
bev_konsern <- konsern_aggregert[!konsern_aggregert$region %in% manglende$region,]

In [ ]:
#sjekk <- bev_konsern %>%
#filter(grepl('EAK',region))

#summary(sjekk)

## Kasse
- Filterer ut en fil for kassetegnskapet og fjerner estimerte kommuner

In [ ]:
bev_kasse <- filter(resultat, regnskapsomfang == "B")

In [ ]:
bev_kasse <- bev_kasse[!bev_kasse$region %in% manglende$region,]

## Konsolidert
- KostraRegnskaps-funksjonen er ikke laget for konsolidert. 
- Konsolidert skal behandles likt som kasse. Mekker derfor til ved å kalle sette regnskapsomfang B på konsolidert og kjøre KostraRegnskap for kun B. 
- Etter kjøringen endres regnskapsomfang tilbake til C. 

In [ ]:
resultat_konsolidert <- KostraRegnskap(data=konsolidert, funksjonshierarki=funksjonshierarki,artshierarki=artshierarki, data_saer=NULL,
  artshierarki_nettinger=NULL,artshierarki_nettinger_kasse=NULL,stjernetabell=stjerne,formler=formler,regnskapsomfang="B")

In [ ]:
resultat_konsolidert$regnskapsomfang <- gsub('B','C',resultat_konsolidert$regnskapsomfang)

## Samler kasse og konsolidert

In [ ]:
bev_kasse_konsolidert <- rbind(bev_kasse, resultat_konsolidert)

# Ferdige filer
- Skriver det ferdig aggregerte konsernregnskapet til mappe

In [ ]:
lagring_konsern <- paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/bev_konsern.parquet', sep = "")

In [ ]:
write_parquet(bev_konsern, lagring_konsern)

- Filtrerer ut kasse-tallene fra resultatet av KostraRegnskaps-funksjonen 
- Skriver kasseregnskapet til mappe

In [ ]:
lagring_kasse_konsolidert <- paste('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/',aar,'/bev_kasse_konsolidert.parquet', sep = "")

In [ ]:
write_parquet(bev_kasse_konsolidert, lagring_kasse_konsolidert)